In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/media/beans/ssd/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import run_epoch, eval_rw

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'8oeshbfn'

In [3]:
wandb.init(id='1h7dizrn', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.18 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

13332.809

In [5]:
m.backbone

EfficientNet(
  (conv_stem): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
        (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(40, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
 

In [6]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [7]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=1e-4)

In [8]:
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

In [9]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs


In [10]:
#wandb.watch(m, log="all")

In [11]:
model_stem = "6.10"
START_E = 58

In [ ]:
n_epochs = 100
log_wandb = True

for e in range(START_E, n_epochs):
    
    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=2650)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 58

{'trn_control_loss': 0.00369586, 'consistency losses/trn_steer_cost': 0.00609303, 'consistency losses/trn_te_loss': 0.02307335, 'consistency losses/trn_%_updates_w_torque_loss': 0.078125, 'consistency losses/trn_%_updates_w_torque_delta_loss': 1.0, 'consistency losses/trn_torque_delta_loss': 43.56669617, 'logistical/obs_consumed_per_second': 241.18110236, 'logistical/obs_generated_per_second': 109.81102362, 'logistical/data_consumption_ratio': 2.20010732, 'logistical/manual_train_pause': 0.0, 'consistency losses/trn_torque_loss': 132.9375, 'logistical/max_param': 16.24702454, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 244.8515625, 'logistical/obs_generated_per_second': 116.734375, 'logistical/data_consumption_ratio': 2.11000716, 'logistical/manual_train_pause': 7.812e-05, 'trn_control_loss': 0.00299527, 'consistency losses/trn_steer_cost': 0.00512162, 'consistency losses/trn_te_loss': 0.01933422, 'consistency losses/trn_%_updates_w_torque_loss': 0.0625, '

{'logistical/obs_consumed_per_second': 239.3203125, 'logistical/obs_generated_per_second': 107.6171875, 'logistical/data_consumption_ratio': 2.22688408, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00278567, 'consistency losses/trn_steer_cost': 0.00490104, 'consistency losses/trn_te_loss': 0.01829516, 'consistency losses/trn_%_updates_w_torque_loss': 0.0234375, 'consistency losses/trn_%_updates_w_torque_delta_loss': 1.0, 'consistency losses/trn_torque_delta_loss': 38.56210327, 'consistency losses/trn_torque_loss': 120.77083333, 'logistical/max_param': 16.24674416, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 239.734375, 'logistical/obs_generated_per_second': 110.40625, 'logistical/data_consumption_ratio': 2.179179, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00288068, 'consistency losses/trn_steer_cost': 0.00513867, 'consistency losses/trn_te_loss': 0.01944681, 'consistency losses/trn_%_updates_w_torque_loss': 0.0390625, 'consistency lo

{'logistical/obs_consumed_per_second': 237.71875, 'logistical/obs_generated_per_second': 115.1953125, 'logistical/data_consumption_ratio': 2.06622106, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00290306, 'consistency losses/trn_steer_cost': 0.00475436, 'consistency losses/trn_te_loss': 0.01810152, 'consistency losses/trn_%_updates_w_torque_loss': 0.078125, 'consistency losses/trn_%_updates_w_torque_delta_loss': 1.0, 'consistency losses/trn_torque_delta_loss': 38.21394348, 'consistency losses/trn_torque_loss': 266.76875, 'logistical/max_param': 16.2436409, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 240.1875, 'logistical/obs_generated_per_second': 103.75, 'logistical/data_consumption_ratio': 2.32310299, 'logistical/manual_train_pause': 7.812e-05, 'trn_control_loss': 0.00354147, 'consistency losses/trn_steer_cost': 0.00497732, 'consistency losses/trn_te_loss': 0.01870056, 'consistency losses/trn_%_updates_w_torque_loss': 0.0390625, 'consistency losses

{'trn_control_loss': 0.00257918, 'consistency losses/trn_steer_cost': 0.00460272, 'consistency losses/trn_te_loss': 0.01748219, 'consistency losses/trn_%_updates_w_torque_loss': 0.0859375, 'consistency losses/trn_%_updates_w_torque_delta_loss': 1.0, 'consistency losses/trn_torque_delta_loss': 36.2406311, 'logistical/obs_consumed_per_second': 239.4015748, 'logistical/obs_generated_per_second': 119.96062992, 'logistical/data_consumption_ratio': 2.00015207, 'logistical/manual_train_pause': 0.0, 'consistency losses/trn_torque_loss': 79.05397727, 'logistical/max_param': 16.24633408, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 246.765625, 'logistical/obs_generated_per_second': 110.4609375, 'logistical/data_consumption_ratio': 2.23571218, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00275388, 'consistency losses/trn_steer_cost': 0.00490934, 'consistency losses/trn_te_loss': 0.01886007, 'consistency losses/trn_%_updates_w_torque_loss': 0.03125, 'consistency l

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s


In [13]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e57.torch")